In [1]:
import numpy as np
import cv2
import os, glob

# path of your data/haarcascades/ folder
haar_data_path = "data/"
    

def generate_images(path):
    tr_images = list(sorted(glob.glob(path + "*.*")))

    face_cascade = cv2.CascadeClassifier(haar_data_path + "haarcascade_frontalface_default.xml")
    count = 1
    for tr_img in tr_images:
        img = cv2.imread(tr_img)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            f = cv2.resize(gray[y:y+h, x:x+w], (200, 200))
            cv2.imwrite(path + 'tr_%s.pgm' % str(count), f)
        count += 1

In [2]:
def read_images(file, sz=(200,200)):
    c = 0
    X,y = [], []
    with open(file) as fp:
        lines = fp.readlines()
        for line in lines:
            line = line.strip()
            tr_image_file,id = line.split(",")
            try:
                img = cv2.imread(tr_image_file, cv2.IMREAD_GRAYSCALE)
                # resize to given size (if given)
                if (sz is not None):
                    img = cv2.resize(img, sz)
                X.append(np.asarray(img, dtype=np.uint8))
                y.append(int(id))
            except:
                print ("Unexpected error:" + sys.exc_info()[0])
                raise
    return [X,y]

In [3]:
def face_recognition(trainfile, testpath, outpath, sz=(200,200)):
    names = ['Alia', 'Taapsee']

    [X,y] = read_images(trainfile)
    y = np.asarray(y, dtype=np.int32)
    #print(y)

    model = cv2.face.EigenFaceRecognizer_create()
    model.train(np.asarray(X), np.asarray(y))
    
    face_cascade = cv2.CascadeClassifier(haar_data_path + "haarcascade_frontalface_default.xml")
    
    test_images = [os.path.basename(x) for x in glob.glob(testpath + '*.*')]
    print(test_images)
    
    for line in test_images:
        line = line.strip()
        img = cv2.imread(testpath+line)
        faces = face_cascade.detectMultiScale(img, 1.3, 5)
        
        for (x, y, w, h) in faces:
            img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            roi = gray[x:x+w, y:y+h]
            try:
                roi = cv2.resize(roi, sz)
                params = model.predict(roi)
                #print ("Label: %s, Confidence: %.2f" % (params[0], params[1]))
                cv2.putText(img, names[params[0]], (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, 255, 2)
            except:
                continue
        cv2.imwrite(outpath+line, img)

In [4]:
generate_images('images/training/')

In [5]:
face_recognition('data.txt', 'images/test/', 'images/classified/')

['taapsee_1.jpeg', 'alia_1.jpeg']
